爬取所有缩略信息页面，并将获得的html字符串保存到文件



In [3]:
# 爬虫 缩略信息页
import time
import requests
def getHTML(url,headers):
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html

total = 5
base_url = "https://wuxi.esf.fang.com/house/i3"
headers = {
    "Referer":"http://search.fang.com/",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183",
    "Cookie":"global_cookie=f4z9z063jq34o28vxq52eephp1cljskp536; __utmc=147393320; csrfToken=P76IEfS5nfQwdr-NqoD9J7P5; city=wuxi; g_sourcepage=esf_fy%5Elb_pc; __utma=147393320.564282534.1688734407.1690616400.1690621484.7; __utmz=147393320.1690621484.7.7.utmcsr=wuxi.esf.fang.com|utmccn=(referral)|utmcmd=referral|utmcct=/house/i31/; __utmt_t0=1; __utmt_t1=1; __utmt_t2=1; unique_cookie=U_142g6osfbe451l5026qi7z4sm18lknpcm91*63; __utmb=147393320.9.10.1690621484"
}
for page in range(1,total+1):
    url = base_url+str(page)+"/"
    html_str = getHTML(url,headers)
    file_name = r"E:\pycharm files\爬虫\房天下二手房定价模型\data\page"+str(page)+".txt"    
    with open(file_name ,"w") as f:
        f.write(html_str)
    print(f"保存第{page}页")
    time.sleep(0.1)

print("over")


保存第1页
保存第2页
保存第3页
保存第4页
保存第5页
over


In [4]:
def create_json(id,url):
    dic = {
            "id":id,
           "name":"小区名称",
           "url":url,
           "price":{"per":"每平方单价",
                "total":"总价",
                "unit":"价格单位"
            },
           "location":{
               "city":"城市",
               "area":"区域",
               "detail":"详细地址",
               "llitude":"经纬度",
               "advantage":{
                   "subway":"地铁"
               }
            },
           "house_info":{
               "square":{
                   "target":"target",
                    "min":"min",
                    "max":"max"
                },
               "type":"3室2厅",
               "floor":{
                   "target":"出售楼层",
                   "total":"总楼层"
               },
               "adornment":"装修",
               "orientation":"朝向"
               ""
           },
        "resource_info":{
            "age":"建筑年代",
            "elevator":"有无电梯",
            "right":"产权性质",
            "house_category":"住宅类别",
            "structure":"建筑结构",
            "build_category":"建筑类别",
            "time":"挂牌时间"
        },
        "cell_info":{
            "refer":"参考均价",
            "tenement":"物业类型",
            "fare":"物业费用",
            "years":"产权年限",
            "green":"绿化率",
            "plot":"容积率",
            "vehicles":"人车分流",
            "build_total":"总栋数",
            "house_total":"总户数"
        },
        "others":{
            "core":"核心卖点",
            "mental":"业主心态",
            "commission":"税费分析",
            "setup":"小区配套",
            "service":"服务介绍"}
    
    }
    return dic

In [5]:
def create_json1(id,title,url):
    dic = {
            "id":id,
           "title":title,
           "url":url
    }
    return dic

In [6]:
def init_xml_code(idx):
    xml_code ={
        "url1":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/h4/a/@href",
        "标题":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/h4/a/span/text()",
        "总价":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[2]/span[1]/b/text()",
        "均价":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[2]/span[2]/text()",
                # /html/body/div[4]/div[4]/div[4]/dl[2]/dd[1]/p[1]/text()[1]
        "户型":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/text()[1]",
        "面积":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/text()[2]",
        "楼层":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/a/text()",
        "总楼层":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/text()[4]",
        "朝向":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/text()[5]",
        "建筑年代":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[1]/text()[6]",
        "小区":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[2]/a/text()",
        "地址":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[2]/span/text()",
        "优势1":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[3]/a/text()",
        "优势2":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[3]/span[1]/text()",
        "优势3":"/html/body/div[4]/div[4]/div[3]/dl[*]/dd[1]/p[3]/span[2]/text()"
    }
    for k,v in xml_code.items():
        xml_code[k] = xml_code[k].replace('*',str(idx))
    return xml_code
type(init_xml_code(2))

dict

In [12]:
# 数据清洗: 从缩略信息中提取出id和url
from lxml import etree
import json

total = 101
info_list = []
flag = 0
for page in range(2,total):
    file_name = r"E:\pycharm files\爬虫\房天下二手房定价模型\data\缩略页\page"+str(page)+".txt"
    with open(file_name,"r") as f:
        html_str = f.read()
    result = etree.HTML(html_str)
    li = result.xpath("/html/body/div[4]/div[4]/div[3]/dl")
    len_li = len(li)
    print(f"第{page}页: {len_li}条数据")
    for idx in range(0,len_li):
        flag = 0
        xml_code = init_xml_code(idx)
        dic = dict()
        for k,v in xml_code.items():
            try:
                r = str(result.xpath(v)[0]).strip()
                dic[k] = r
            except Exception as e:
                dic[k] = []
                if(k=="url1"):
                    flag = 1
            # print(f"{k}: {dic[k]}")
        if flag==0:
            info_list.append(dic)
    print("\n")
                

info_filename = r"E:\pycharm files\爬虫\房天下二手房定价模型\data\sketchy_info.json"
with open(info_filename,"w") as f:
    json_str = json.dumps(info_list)
    f.write(json_str)
#     
    
print("over")


第2页: 60条数据


第3页: 60条数据


第4页: 60条数据


第5页: 60条数据


第6页: 60条数据


第7页: 60条数据


第8页: 60条数据


第9页: 60条数据


第10页: 60条数据


第11页: 60条数据


第12页: 60条数据


第13页: 60条数据


第14页: 60条数据


第15页: 60条数据


第16页: 60条数据


第17页: 60条数据


第18页: 60条数据


第19页: 60条数据


第20页: 60条数据


第21页: 60条数据


第22页: 60条数据


第23页: 60条数据


第24页: 60条数据


第25页: 60条数据


第26页: 60条数据


第27页: 60条数据


第28页: 60条数据


第29页: 60条数据


第30页: 60条数据


第31页: 60条数据


第32页: 60条数据


第33页: 60条数据


第34页: 60条数据


第35页: 60条数据


第36页: 60条数据


第37页: 60条数据


第38页: 60条数据


第39页: 60条数据


第40页: 60条数据


第41页: 60条数据


第42页: 60条数据


第43页: 60条数据


第44页: 60条数据


第45页: 60条数据


第46页: 60条数据


第47页: 60条数据


第48页: 60条数据


第49页: 60条数据


第50页: 60条数据


第51页: 60条数据


第52页: 60条数据


第53页: 60条数据


第54页: 60条数据


第55页: 60条数据


第56页: 60条数据


第57页: 60条数据


第58页: 60条数据


第59页: 60条数据


第60页: 60条数据


第61页: 60条数据


第62页: 60条数据


第63页: 60条数据


第64页: 60条数据


第65页: 60条数据


第66页: 60条数据


第67页: 60条数据


第68页: 60条数据


第69页: 60条数据


第70页: 60条数据


第71页: 60条数据


第72页: 60条数据


第73页: 60条数据




id:小区编号
name:小区名称
url:链接地址
price:{per:每平方单价,total:总价,unit:价格单位}价格
location:{city:城市,area:区域,detail:详细地址,llitude:经纬度,advantage:优势{subway:地铁}}地址
house_info:房屋信息{square:面积{target,min,max},type:3室2厅,floor:楼层{target:出售楼层,total:总楼层},adornment:装修,orientation:朝向}
resource_info:房源信息{age:建筑年代,elevator:有无电梯,right:产权性质,house_category:住宅类别,structure:建筑结构,build_category:建筑类别,time:挂牌时间}
cell_info:小区信息{refer:参考均价,tenement:物业类型,green:绿化率,plot:容积率,vehicles:人车分流,build_total总栋数,house_total:总户数}
others:其他{core:核心卖点,mental:业主心态,commission:税费分析,setup:小区配套,service:服务介绍}

In [8]:
import json

j_str = json.dumps(dic)
j = json.loads(j_str)
print(j)
j["name"]

info_list = []

{'id': '3_172883474', 'name': '小区名称', 'url': 'https://wuxi.esf.fang.com/chushou/3_172883474.htm', 'price': {'per': '每平方单价', 'total': '总价', 'unit': '价格单位'}, 'location': {'city': '城市', 'area': '区域', 'detail': '详细地址', 'llitude': '经纬度', 'advantage': {'subway': '地铁'}}, 'house_info': {'square': {'target': 'target', 'min': 'min', 'max': 'max'}, 'type': '3室2厅', 'floor': {'target': '出售楼层', 'total': '总楼层'}, 'adornment': '装修', 'orientation': '朝向'}, 'resource_info': {'age': '建筑年代', 'elevator': '有无电梯', 'right': '产权性质', 'house_category': '住宅类别', 'structure': '建筑结构', 'build_category': '建筑类别', 'time': '挂牌时间'}, 'cell_info': {'refer': '参考均价', 'tenement': '物业类型', 'fare': '物业费用', 'years': '产权年限', 'green': '绿化率', 'plot': '容积率', 'vehicles': '人车分流', 'build_total': '总栋数', 'house_total': '总户数'}, 'others': {'core': '核心卖点', 'mental': '业主心态', 'commission': '税费分析', 'setup': '小区配套', 'service': '服务介绍'}}


In [84]:
import requests

base_url = "https://wuxi.esf.fang.com/chushou/3_172846038.htm"
headers = {
    "Referer":"http://search.fang.com/",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183",
    "Cookie":"global_cookie=f4z9z063jq34o28vxq52eephp1cljskp536; __utmc=147393320; csrfToken=P76IEfS5nfQwdr-NqoD9J7P5; city=wuxi; g_sourcepage=esf_fy%5Elb_pc; __utma=147393320.564282534.1688734407.1690616400.1690621484.7; __utmz=147393320.1690621484.7.7.utmcsr=wuxi.esf.fang.com|utmccn=(referral)|utmcmd=referral|utmcct=/house/i31/; __utmt_t0=1; __utmt_t1=1; __utmt_t2=1; unique_cookie=U_142g6osfbe451l5026qi7z4sm18lknpcm91*63; __utmb=147393320.9.10.1690621484"
}
resp = requests.get(url,headers=headers)
html_str = resp.text

In [86]:
# print(html_str)

In [ ]:
xml_code = {
    "户型":"/html/body/div[4]/div[1]/div[4]/div[3]/div[1]/div[1]",
    "建筑面积":"/html/body/div[4]/div[1]/div[4]/div[3]/div[2]/div[1]",
    "单价":"/html/body/div[4]/div[1]/div[4]/div[3]/div[3]/div[1]",
    "朝向":"/html/body/div[4]/div[1]/div[4]/div[4]/div[1]/div[1]",
    "楼层":"/html/body/div[4]/div[1]/div[4]/div[4]/div[2]/div[1]/a",
    "总楼层":"/html/body/div[4]/div[1]/div[4]/div[4]/div[2]/div[2]",
    "装修":"/html/body/div[4]/div[1]/div[4]/div[4]/div[3]/div[1]/a",
    "小区":"/html/body/div[4]/div[1]/div[4]/div[5]/div[1]/div[2]/a[1]",
    "区域":"/html/body/div[4]/div[1]/div[4]/div[5]/div[2]/div[2]/a[1]",
    "建筑年度":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[1]/span[2]",
    "有无电梯":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[2]/span[2]",
    "产权性质":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[3]/span[2]/a",
    "住宅类型":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[4]/span[2]/a",
    "建筑结构":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[5]/span[2]/a",
    "建筑类别":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[6]/span[2]/a",
    "挂牌日期":"/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/div[7]/span[2]",
    "参考均价":"/html/body/div[4]/div[2]/div[1]/div[4]/div[2]/div[1]/div[1]/span[2]/i",
    "物业类型":"/html/body/div[4]/div[2]/div[1]/div[4]/div[2]/div[2]/div[1]/span[2]",
    "物业费用":"/html/body/div[4]/div[2]/div[1]/div[4]/div[2]/div[2]/div[2]/span[2]",
    "产权年限":"/html/body/div[4]/div[2]/div[1]/div[4]/div[2]/div[2]/div[4]/span[2]",
    "":"",
    "":"",
    "":"",
    "":"",
    "":"",
}
